In [1]:
import pandas as pd
import random
import csv

# 1. Recommendation System with LSH

## 1.1 Data Preparation

Before we biggin let's download and explore our dataset.

In [2]:
# Loading all the data
movies_df = pd.read_csv('movie.csv')
g_scores_df = pd.read_csv('genome_scores.csv')
g_tags_df = pd.read_csv('genome_tags.csv')
link_df = pd.read_csv('link.csv')
rating_df = pd.read_csv('rating.csv')
tag_df = pd.read_csv('tag.csv')

For this part of the project we will need only the movies and rating df. So we are going to merge them and analyse them.

In [3]:
# Merge the two Data sets
titles_and_ratings_df = pd.merge(movies_df, rating_df) 

In [12]:
titles_and_ratings_df

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5,2009-01-02 01:13:41
...,...,...,...,...,...,...
20000258,131254,Kein Bund für's Leben (2007),Comedy,79570,4.0,2015-03-30 19:32:59
20000259,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,79570,4.0,2015-03-30 19:48:08
20000260,131258,The Pirates (2014),Adventure,28906,2.5,2015-03-30 19:56:32
20000261,131260,Rentun Ruusu (2001),(no genres listed),65409,3.0,2015-03-30 19:57:46


In [15]:
# Checking for missing values
titles_and_ratings_df.isnull().sum()

movieId      0
title        0
genres       0
userId       0
rating       0
timestamp    0
dtype: int64

In [16]:
titles_and_ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   title      object 
 2   genres     object 
 3   userId     int64  
 4   rating     float64
 5   timestamp  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 915.5+ MB


In [ ]:
# Total number of movies in the data set
len(titles_and_ratings_df["title"].unique())

26729

In [24]:
# All the kinds of genres
pd.DataFrame(titles_and_ratings_df["genres"].unique(), columns= ["Kinds"])

,Kinds
0,Adventure|Animation|Children|Comedy|Fantasy
1,Adventure|Children|Fantasy
2,Comedy|Romance
3,Comedy|Drama|Romance
4,Comedy
...,...
1324,Adventure|Children|Drama|Sci-Fi
1325,Children|Documentary|Drama
1326,Action|Adventure|Animation|Fantasy|Horror
1327,Animation|Children|Comedy|Fantasy|Sci-Fi


In [25]:
# Total number of users
len(titles_and_ratings_df["userId"].unique())

138493

## 1.2 Minhash Signatures

Using the <strong> userId </strong> and <strong> movieId </strong> columns, implement your own MinHash function. This function will hash each user's watched movie list, creating a representation that allows for quick comparisons of user similarities.

To start with, we are going to create a dictionary that will contain all the movies each user has watched. So as keys we are going to use userId and as values, the movieId.

In [4]:
# Create the dictionary
users_dict = titles_and_ratings_df.groupby('userId')['movieId'].apply(set).to_dict()

Now we can move on and build our own MinHash function.

In [5]:
# First we have to define the hash function
def hash_function(hashes, values, prime):
    # It creates a number of hash functions and puts them in a list
    hashes_list = []
    for i in range(hashes):
        a = random.randint(1, values)
        b = random.randint(0, values)
        hashes_list.append(lambda x, a=a, b=b, p=prime: (a * x + b) % p)
    return hashes_list

In [6]:
# Define MinHash Function
def minhash(movie_set, hashes_list):
    minhash_vector = []
    for i in hashes_list:
        min_hash = min(i(title) for title in movie_set)
        minhash_vector.append(min_hash)
    return minhash_vector

After building our own MinHash function, we are going to define the number of hashes as well as the maximum values (values), in order to generate signature vectors for each user based on their rated movies

In [15]:
# Number of hash functions and values 
hashes = 150
prime = 150001
values = max(titles_and_ratings_df['movieId'])

hashes_list = hash_function(hashes, values, prime)

# Save each user's signature in a dictionary
users_signatures = {}
for userid, movie_set in users_dict.items():
    minhash_vector = minhash(movie_set, hashes_list)
    users_signatures[userid] = minhash_vector

In [22]:
users_signatures

{1: [348,
  804,
  1000,
  169,
  1495,
  3217,
  167,
  1059,
  1025,
  189,
  56,
  1228,
  11,
  720,
  257,
  403,
  201,
  2086,
  356,
  2888,
  1,
  564,
  266,
  210,
  393,
  508,
  2170,
  889,
  581,
  401,
  364,
  883,
  3640,
  3145,
  980,
  87,
  660,
  976,
  597,
  932,
  1222,
  118,
  16,
  60,
  1848,
  577,
  1552,
  3375,
  2483,
  158,
  1755,
  995,
  2166,
  921,
  955,
  839,
  0,
  3365,
  2365,
  1300,
  545,
  431,
  682,
  493,
  401,
  87,
  1232,
  340,
  549,
  218,
  614,
  1140,
  2033,
  128,
  1366,
  1178,
  51,
  2049,
  131,
  194,
  1396,
  4012,
  344,
  321,
  467,
  2062,
  677,
  54,
  192,
  7,
  1567,
  724,
  312,
  228,
  138,
  379,
  591,
  1331,
  1058,
  318,
  222,
  1489,
  582,
  11,
  463,
  2410,
  316,
  595,
  2189,
  19,
  839,
  2285,
  75,
  45,
  2303,
  870,
  382,
  40,
  558,
  22,
  2085,
  375,
  289,
  831,
  688,
  5540,
  2167,
  1402,
  5544,
  1023,
  3228,
  137,
  77,
  277,
  401,
  117,
  565,
  1770,
  2165

In [20]:
# Save it in A csv file
csv_filename = 'users_signatures.csv'

# Open the file in write mode
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header 
    writer.writerow(['User', 'Signature'])
    
    # Write the dictionary data
    for user, signature in users_signatures.items():
        # Join the signature list into a string
        signature_str = ','.join(map(str, signature))
        # Write each row
        writer.writerow([user, signature_str])

Now if we want to do quick comparisons of user similarities, we have to create a <strong> Jaccard similarity </strong> function.

In [16]:
# Define Jaccard Similarity function
def jaccard_similarity(user1, user2):
    # user1 = signature of the first user
    # user2 = signature of the second user
    similarity = sum(1 for a, b in zip(user1, user2) if a == b) / len(user1)
    return similarity

In [17]:
# Testing 
user1 = users_signatures[10]
user2 = users_signatures[10]
similarity = jaccard_similarity(user1, user2)
print("The similarity between the 2 users is: ", similarity)

The similarity between the 2 users is:  1.0


Experiment with different hash functions and threshold values to find the most effective configurations. Report these results.

## 1.3 Locality-Sensitive Hashing (LSH)

First we have to convert the signatures column into a list of integers:

2. Define the LSH Function
The LSH function remains the same, but now signatures is a list of lists:

In [18]:
import hashlib
from collections import defaultdict

def lsh(users_signatures, num_bands, rows_per_band):
    """
    Apply Locality-Sensitive Hashing (LSH) to cluster similar users using a dictionary.
    :param user_signatures: Dictionary {userId: [signature]}.
    :param num_bands: Number of bands to divide the signature into.
    :param rows_per_band: Number of rows per band.
    :return: Dictionary of buckets {band_hash: [userIds]}.
    """
    buckets = defaultdict(list)  # Dictionary to store buckets
    num_hashes = len(next(iter(users_signatures.values())))  # Get length of the first signature
    assert num_hashes == num_bands * rows_per_band, \
        "Number of hash values must equal num_bands * rows_per_band"

    for user_id, signature in users_signatures.items():
        for band in range(num_bands):
            # Extract the rows for this band
            start = band * rows_per_band
            end = start + rows_per_band
            band_values = tuple(signature[start:end])

            # Hash the band values to form a bucket key
            band_hash = hashlib.md5(str(band_values).encode('utf-8')).hexdigest()

            # Add user ID to the corresponding bucket
            buckets[band_hash].append(user_id)
    
    return buckets



3. Apply LSH
For 
𝑛
=
150
,
𝑏
=
30
,
𝑟
=
5
n=150,b=30,r=5:

In [24]:
# Set parameters
num_bands = 30
rows_per_band = 5

# Apply LSH
buckets = lsh(users_signatures, num_bands, rows_per_band)

# Inspect a few buckets
for bucket_hash, users in list(buckets.items())[:5]:  # Check the first 5 buckets
    print(f"Bucket {bucket_hash}: Users {users}")

Bucket 3ad3e75ffa00745918ee2b75f831c12e: Users [1]
Bucket 60163ae646ab6de1fbf9128738b594e3: Users [1]
Bucket d971b8c633f20941d71cd5de85eab31d: Users [1]
Bucket aebc4c1a93687b924c74f708c6475c17: Users [1]
Bucket bc8ce915c38a3d2baa3cd0b9f51198e8: Users [1]


In [25]:
# Create clusters
clusters = [users for users in buckets.values() if len(users) > 1]
print(f"Total clusters found: {len(clusters)}")
for cluster in clusters[:5]:  # Display the first 5 clusters
    print(f"Cluster: {cluster}")

Total clusters found: 208057
Cluster: [1, 70996]
Cluster: [1, 130035]
Cluster: [2, 21944]
Cluster: [3, 66047, 81290, 115541]
Cluster: [3, 4554, 14261, 50142, 91570, 92016, 92419, 100848]


Now, for a given user, identify the two most similar users based on their bucket placement. If a user doesn’t have any similar users in their bucket, adjust the parameters until similar users are found

In [26]:
def find_most_similar_users(user_id, buckets, users_signatures, jaccard_similarity):
    # Step 1: Get the list of users in the same bucket as the given user
    similar_users = []
    
    # Look through all the buckets to find the one containing the given user
    for bucket_hash, users_in_bucket in buckets.items():
        if user_id in users_in_bucket:
            similar_users = users_in_bucket
            break
    
    # Step 2: If no similar users were found, return a tuple with None values
    if len(similar_users) <= 1:
        return None, None  # We return None for both values
    
    # Step 3: Compare the users in the same bucket using Jaccard similarity
    most_similar_users = []
    max_similarity = 0
    
    # Loop over all pairs of users in the same bucket
    for user1_id in similar_users:
        if user1_id == user_id:
            continue
        for user2_id in similar_users:
            if user1_id == user2_id or user2_id == user_id:
                continue
            # Compute similarity between user1_id and user2_id using your Jaccard function
            similarity = jaccard_similarity(users_signatures[user1_id], users_signatures[user2_id])
            if similarity > max_similarity:
                max_similarity = similarity
                most_similar_users = [user1_id, user2_id]
    
    return most_similar_users, max_similarity


In [27]:
# Example usage for a specific user (e.g., user 158)
user_id = 158
most_similar_users, max_similarity = find_most_similar_users(user_id, buckets, users_signatures, jaccard_similarity)
print(f"The two most similar users to user {user_id} are: {most_similar_users} with a similarity of {max_similarity}")

The two most similar users to user 158 are: [9905, 90063] with a similarity of 0.8333333333333334


Movie Recommendation Logic:

If both similar users have rated a movie, recommend this movie based on the average rating.
If there are no commonly rated movies, recommend the top-rated movies of the most similar user.

In [67]:
del set

In [31]:
def recommend_movies(user_id, similar_users, rating_df, movies_df, users_signatures):
    """
    Recommend movies for a given user based on the most similar users and their ratings, including movie titles.
    
    Returns:
        A DataFrame with recommended movies and corresponding information.
    """
    user_recommendations = []
    similar_user1, similar_user2 = similar_users

    # Step 1: Get ratings for the similar users
    user1_ratings = rating_df[rating_df['userId'] == similar_user1]
    user2_ratings = rating_df[rating_df['userId'] == similar_user2]

    # Step 2: Find common movies rated by both similar users
    user1_movies = set(user1_ratings['movieId'])
    user2_movies = set(user2_ratings['movieId'])
    common_movies = user1_movies.intersection(user2_movies)

    if common_movies:
        # Add common movies with average ratings
        for movie in common_movies:
            rating1 = user1_ratings[user1_ratings['movieId'] == movie]['rating'].values[0]
            rating2 = user2_ratings[user2_ratings['movieId'] == movie]['rating'].values[0]
            avg_rating = (rating1 + rating2) / 2

            movie_title = movies_df[movies_df['movieId'] == movie]['title'].values[0]
            user_recommendations.append({'Target User': user_id, 
                                         'Similar User': similar_user1, 
                                         'Movie ID': movie, 
                                         'Title': movie_title, 
                                         'Rating': avg_rating})
            user_recommendations.append({'Target User': user_id, 
                                         'Similar User': similar_user2, 
                                         'Movie ID': movie, 
                                         'Title': movie_title, 
                                         'Rating': avg_rating})

    # Step 3: Handle the case when fewer than 5 movies are found
    if len(user_recommendations) < 5:
        # Calculate Jaccard similarity between the target user and the similar users
        target_user_signature = users_signatures[user_id]
        similarity_user1 = jaccard_similarity(target_user_signature, users_signatures[similar_user1])
        similarity_user2 = jaccard_similarity(target_user_signature, users_signatures[similar_user2])

        # Choose the most similar user
        most_similar_user = similar_user1 if similarity_user1 > similarity_user2 else similar_user2

        # Add top-rated movies from the most similar user
        most_similar_ratings = rating_df[rating_df['userId'] == most_similar_user]
        top_rated_movies = most_similar_ratings.sort_values(by='rating', ascending=False)

        # Avoid recommending already included movies
        already_recommended = {rec['Movie ID'] for rec in user_recommendations}
        for _, row in top_rated_movies.iterrows():
            if len(user_recommendations) >= 5:
                break
            if row['movieId'] not in already_recommended:
                movie_title = movies_df[movies_df['movieId'] == row['movieId']]['title'].values[0]
                user_recommendations.append({'Target User': user_id, 
                                             'Similar User': most_similar_user, 
                                             'Movie ID': row['movieId'], 
                                             'Title': movie_title, 
                                             'Rating': row['rating']})

    # Step 4: Finalize the recommendations
    recommendations_df = pd.DataFrame(user_recommendations).head(5)
    return recommendations_df



In [36]:
# Example usage: Recommend movies for user 1 based on most similar users [2, 3]
user_id = 6
similar_users = [1547, 476]
recommended_movies = recommend_movies(user_id, similar_users, rating_df, movies_df, users_signatures)

print(recommended_movies)

   Target User  Similar User  Movie ID                               Title  \
0            6          1547       357  Four Weddings and a Funeral (1994)   
1            6           476       357  Four Weddings and a Funeral (1994)   
2            6          1547         7                      Sabrina (1995)   
3            6           476         7                      Sabrina (1995)   
4            6          1547       648          Mission: Impossible (1996)   

   Rating  
0     3.5  
1     3.5  
2     4.5  
3     4.5  
4     4.0  
